In [1]:
import numpy as np

# Problem No. 01

## MaxMatch

In [2]:
def Max_Size(words_dict):
    '''
        returns max size and index of word in 'words_dict' 
    '''
    max_size = 0
    max_index = 0
    for i in range(0, len(words_dict)):
        if (len(words_dict[i]) > max_size):
            max_size = len(words_dict[i])
            max_index = i
    return max_size, max_index

In [3]:
def Match_Word(word, dict_):
    '''
        return True if 'word' is found in 'dict_'
    '''
    match = False
    
    for i in range(0, len(dict_)):
        if(word == dict_[i]):
            match = True
            break
    
    return match

In [4]:
def MaxMatch (sentence, words_dict):
    '''
    Input:
        sentence:   a string that is to be tokenized using max match algorithm
        words_dict: the list of words i.e. dictionary
    Output:
        words_match_list:  The list of words extracted after applying max match
    '''
    tokenized_sentence = None
    max_word_size, max_word_indx = Max_Size(words_dict)
    
    words_match_list = []
    
    # print('Max_Word_Size:\t\t'+str(max_word_size)+"\t\t Max_Size_Word:\t\t"+str(words_dict[max_word_indx]) + "\tindex:\t\t" + str(max_word_indx))
    
    a = 0
    for i in range(0, len(sentence)):
        if(a < len(sentence)):
            flag_match_found = False
            max_word = None
            match_idx = 0

            for j in range(0, max_word_size):

                itr_word = sentence[a:(a+j+1)]

                if(Match_Word(itr_word, words_dict)):
                    max_word = itr_word
                    flag_match_found = True
                    match_idx = j

            if (flag_match_found):
                words_match_list.append(max_word)
                a = a+match_idx+1
            else:
                words_match_list.append(sentence[a])
                a += 1
        else:
            break
    
    return words_match_list

## Minimum Edit Distance

In [5]:
def MED(source, target):
    '''
    input:
        source: a string
        target: a string
    output:
        out: a dict object which contain following items
            'D': The number of deletions required to convert source to target
            'I': The number of insertions required to convert source to target
            'S': The number of substitutions required to convert source to target
            'MED': The minimum edit distance to convert source to target
    '''
    i_cost = 1
    d_cost = 1
    s_cost = 2
    
    n = len(source)
    m = len(target)
    
    Dist = np.zeros((n+1, m+1, 1)).astype('uint8') # n,m,0 = total cost;
    Ops = np.zeros((n+1, m+1, 3)).astype('uint8') # n,m,1 = # of insertions; n,m,0 = # of deletions; n,m,2 = # of substitutions
    
    #initilizations
    for i in range(1, n+1):
        Dist[i, 0, 0] = Dist[i-1, 0, 0] + d_cost
        Ops[i, 0, 0] = 1
    
    for i in range(1, m+1):
        Dist[0, i, 0] = Dist[0, i-1, 0] + i_cost
        Ops[0, i, 1] = 1
        
    #recurrence
    for i in range(1, n+1):
        for j in range(1, m+1):
            costs = [0]*3
            costs[0] = Dist[i-1, j, 0] + d_cost #deletion
            costs[1] = Dist[i, j-1, 0] + i_cost #insertion
            
            if(source[i-1] == target[j-1]): #sub
                costs[2] = Dist[i-1, j-1, 0]
            else:
                costs[2] = Dist[i-1, j-1, 0] + s_cost
            
            Dist[i, j, 0] = min(costs)
            
            minval = min(costs)
            for a in range(0, len(costs)):
                if (costs[a] == minval):
                    Ops[i, j, a] = 1
                    #print(i,j)
            
    #print(Dist)
    #i = 4
    #print(Ops[n-i, m-i])
    #print(Ops[n-i-1, m-i])
    #print(Ops[n-i, m-i-1])
    #print(Ops[n-i-1, m-i-1])
    
    #back-track
    a = n
    b = m
    
    t_s = 0
    t_d = 0
    t_i = 0
    
    while((a != 0) or (b != 0)):
        #print(a, b, Dist[a, b, 0], Ops[a, b])
        costs = []
        op = []
        if(Ops[a, b, 2] == 1):
            costs.append(Dist[a, b, 0])
            op.append("s")
        if(Ops[a, b, 0] == 1):
            costs.append(Dist[a, b, 0])
            op.append("d")
        if(Ops[a, b, 1] == 1):
            costs.append(Dist[a, b, 0])
            op.append("i")
        
        minval = min(costs)
        indx = costs.index(minval)
        #print(indx, costs)
        if (op[indx] == 's'):
            if(Dist[a, b, 0] != Dist[a-1, b-1, 0]):
                t_s += 1
                #print(Dist[a, b, 0], Dist[a-1, b-1, 0])
            a -=1
            b -=1
        elif (op[indx] == 'd'):
            a -=1
            t_d += 1
        elif (op[indx] == 'i'):
            b -=1
            t_i += 1
    
    out = {
        'D': t_d,
        'I': t_i,
        'S': t_s,
        'MED': Dist[n, m, 0]
    }
    
    return out

In [6]:
def char_based (sentence1, sentence2):
    '''
    input:
        sentence1: a string
        sentence2: a string
    output:
        out: a dict object which contain following items
            'total_D': The number of deletions required to convert sentence1 to sentence2
            'total_I': The number of insertions required to convert sentence1 to sentence2
            'total_S': The number of substitutions required to convert sentence1 to sentence2
            'total_MED': The minimum edit distance to convert sentence1 to sentence2
    '''
    len1 = len(sentence1)
    len2 = len(sentence2)
    
    if (len1 > len2):
        len_s = len2
        len_h = len1
    elif(len2 > len1):
        len_s = len1
        len_h = len2
    else:
        len_s = len1
        len_h = len2
    
    total_dist = 0
    total_sub = 0
    total_ins = 0
    total_del = 0
    for i in range(0, len_s):
        #print(i)
        outstruc = MED(sentence1[i], sentence2[i])
        total_dist += outstruc['MED']
        total_sub += outstruc['S']
        total_ins += outstruc['I']
        total_del += outstruc['D']
    
    if(len_s != len_h):
        if(len2 == len_s):
            #delete remaining words of sentence1
            words_toremove = sentence1[len2:]
        else:
            #delete remaining words of sentence2
            words_toremove = sentence2[len1:]
        for i in words_toremove:
            total_dist += len(i)
            total_del += len(i) #we can use insertion as well, our choice
    
    out = {
        'total_MED': total_dist,
        'total_I': total_ins,
        'total_S': total_sub,
        'total_D': total_del
    }
    
    return out

In [7]:
def WER(S, D, I, N):
    '''
    input:
        S: number of substitutions
        D: number of deletions
        I: number of insertions
        N: number of words in correct sentence
    '''
    return (S+D+I)/N

In [8]:
def word_based(sentence1, sentence2): #assuming sentence2 is the target sentence, i.e. correct sentence
    '''
    input:
        sentence1: a string
        sentence2: a string
    output:
        out: a dict object which contain following items
            'D': The number of deletions required to convert sentence1 to sentence2
            'I': The number of insertions required to convert sentence1 to sentence2
            'S': The number of substitutions required to convert sentence1 to sentence2
            'MED': The minimum edit distance to convert sentence1 to sentence2
            'WER': Word error rate
    '''
    out = MED(sentence1, sentence2)
    out['WER'] = WER(out['S'], out['D'], out['I'], len(sentence2))
    return out

In [9]:
def avg_WER(source_set, target_set):
    '''
    input:
        source_set: the list of strings containing the source strings
        target_set: the list of strings containing the target strings
    output:
        avg: the average of Word error rate for whole input set
    '''
    wer = 0
    for i in range(0, len(source_set)):
        wer += word_based(source_set[i], target_set[i])['WER']
    
    avg = wer/len(source_set)
    return avg

# Problem No. 02

## Str2Vec

In [10]:
def str2vec_word_based_count(str1, str2):
    '''
    input:
        
    '''
    vec_words = list(set(str1+str2))
    
    vec1 = [0] * len(vec_words)
    vec2 = [0] * len(vec_words)
    for i in str1:
        vec1[vec_words.index(i)] += 1
    
    for i in str2:
        vec2[vec_words.index(i)] += 1
    
    return vec1, vec2, vec_words

In [11]:
def str2vec_word_based_freq(str1, str2):
    vec1, vec2, vec_words = str2vec_word_based_count(str1, str2)
    
    return (np.array(vec1)/len(str1)).tolist(), (np.array(vec2)/len(str2)).tolist(), vec_words

In [12]:
def str2vec_char_based_count(str1, str2):
    
    vec_words = list(set(str1+str2))
    vec_chars = []
    for i in range(0, len(vec_words)):
        for j in vec_words[i]:
            vec_chars.append(j)
    vec_chars = list(set(vec_chars))
    
    vec1 = [0] * len(vec_chars)
    vec2 = [0] * len(vec_chars)
    for i in range(0, len(str1)):
        for j in str1[i]:
            vec1[vec_chars.index(j)] += 1
    
    for i in range(0, len(str2)):
        for j in str2[i]:
            vec2[vec_chars.index(j)] += 1
    
    
    return vec1, vec2, vec_chars

In [13]:
def str2vec_char_based_freq(str1, str2):
    vec1, vec2, vec_words = str2vec_char_based_count(str1, str2)
    
    return (np.array(vec1)/len(str1)).tolist(), (np.array(vec2)/len(str2)).tolist(), vec_words

In [14]:
def to_bigram(str_):
    bigram = []
    
    if ( str(type(str_)) == "<class 'list'>"):
        for i in range(0, len(str_)):
            if i == 0:
                bigram.append("-"+str_[0][0])
            else:
                bigram.append(str_[i-1][len(str_[i-1])-1]+str_[i][0])
            for j in range(1, len(str_[i])):
                bigram.append(str_[i][j-1]+str_[i][j])
        bigram.append(str_[len(str_)-1][len(str_[len(str_)-1])-1]+"-")
            
    elif(str(type(str_)) == "<class 'str'>"):
        bigram.append("-"+str_[0])
        for j in range(1, len(str_)):
                bigram.append(str_[j-1]+str_[j])
        bigram.append(str_[len(str_)-1]+"-")
        
    return bigram

## Distances

In [15]:
def validation(str1, str2, base, measure):
    if (base == 'char'):
        if (measure == 'freq'):
            return str2vec_char_based_freq(str1, str2)
        elif (measure == 'count'):
            return str2vec_char_based_count(str1, str2)
        else:
            assert False, "measure value UnExpected"
            
    elif (base == 'word'):
        if (measure == 'freq'):
            return str2vec_word_based_freq(str1, str2)
        elif (measure == 'count'):
            return str2vec_word_based_count(str1, str2)
        else:
            assert False, "measure value UnExpected"
    else:
        assert False, "base value UnExpected"
    

In [16]:
def euclidean_distance(str1, str2, base = 'char', measure = 'freq'):
    '''
    input:
        str1: the text string
        str2: the text string
        base: the base on which vector will be made, char/word
        measure: the measure on which the values will be filled in vector, count/freq
    output:
        the computed distance
    '''
    vec1, vec2, vec_list = validation(str1, str2, base, measure)
    
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    
    return np.sqrt(np.sum(np.power((vec1-vec2), 2))).tolist()

In [17]:
def manhattan_distance(str1, str2, base = 'char', measure = 'freq'):
    '''
    input:
        str1: the text string
        str2: the text string
        base: the base on which vector will be made, char/word
        measure: the measure on which the values will be filled in vector, count/freq
    output:
        the computed distance
    '''
    vec1, vec2, vec_list = validation(str1, str2, base, measure)
    
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    
    return np.sum(np.absolute(vec1-vec2)).tolist()

In [18]:
def chebychev_distance(str1, str2, base = 'char', measure = 'freq'):
    '''
    input:
        str1: the text string
        str2: the text string
        base: the base on which vector will be made, char/word
        measure: the measure on which the values will be filled in vector, count/freq
    output:
        the computed distance
    '''
    vec1, vec2, vec_list = validation(str1, str2, base, measure)
    
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    sub = np.absolute(vec1-vec2)
    
    return np.amax(sub)

In [19]:
def cosine_similarity(str1, str2, base = 'char', measure = 'freq'):
    '''
    input:
        str1: the text string
        str2: the text string
        base: the base on which vector will be made, char/word
        measure: the measure on which the values will be filled in vector, count/freq
    output:
        the computed similarity
    '''
    vec1, vec2, vec_list = validation(str1, str2, base, measure)
    
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    dot_ = np.dot(vec1, vec2.T)
    similarity = dot_/((np.sqrt(np.sum(np.power(vec1, 2)))) * (np.sqrt(np.sum(np.power(vec2, 2)))))
    
    return similarity

In [20]:
def cosine_distance(str1, str2, base = 'char', measure = 'freq'):
    '''
    input:
        str1: the text string
        str2: the text string
        base: the base on which vector will be made, char/word
        measure: the measure on which the values will be filled in vector, count/freq
    output:
        the computed distance
    '''
    return 1 - cosine_similarity(str1, str2, base, measure)

In [21]:
def jaccard_similarity(str1, str2, base = 'char'):
    '''
    input:
        str1: the text string
        str2: the text string
        base: the base on which vector will be made, char/word/bigram
    output:
        the computed similarity
    '''
    vec1 = []
    vec2 = []
    if (base == 'char'):
        if ( str(type(str1)) == "<class 'list'>"):
            strtmp = ''
            for i in str1:
                strtmp += i
            vec1 = set(strtmp)
        elif (str(type(str1)) == "<class 'str'>"):
            vec1 = set(str1)
        else:
            assert False, "Garbarrrr"

        if ( str(type(str2)) == "<class 'list'>"):
            strtmp = ''
            for i in str2:
                strtmp += i
            vec2 = set(strtmp)
        elif (str(type(str2)) == "<class 'str'>"):
            vec2 = set(str2)
        else:
            assert False, "Garbarrrr"
    elif (base == 'word'):
        if ( str(type(str1)) == "<class 'list'>"):
            vec1 = set(str1)
        elif ( str(type(str1)) == "<class 'str'>"):
            vec1 = {str1}
        else:
            assert False, "Garbarrrr"
        
        if ( str(type(str2)) == "<class 'list'>"):
            vec2 = set(str2)
        elif ( str(type(str2)) == "<class 'str'>"):
            vec2 = {str2}
        else:
            assert False, "Garbarrrr"
    elif (base == 'bigram'):
        vec1 = set(to_bigram(str1))
        vec2 = set(to_bigram(str2))
    else:
        assert False, "Garbarrrr"
    
    return len(vec1&vec2) / len(vec1|vec2)

In [22]:
def jaccard_distance(str1, str2, base = 'char'):
    '''
    input:
        str1: the text string
        str2: the text string
        base: the base on which vector will be made, char/word/bigram
    output:
        the computed distance
    '''
    return 1 - jaccard_similarity(str1, str2, base)

In [23]:
def hamming_same_length_str(str1, str2):
    count = 0
    for i in range(0, len(str1)):
        if (str1[i] != str2[i]):
            count += 1
    return count    

In [24]:
def hamming_less_length_str(str1, str2):
    count_list = []
    diff = len(str2) - len(str1)
    for i in range(0, diff):
        tempstr = " "*i
        tempstr += str1
        tempstr += " "*(diff-i)
        count_list.append(hamming_same_length_str(tempstr, str2))
    return min(count_list)

In [25]:
def hamming_str(str1, str2):
    if (len(str1) == len(str2)):
        return hamming_same_length_str(str1, str2)
    elif (len(str1) < len(str2)):
        return hamming_less_length_str(str1, str2)
    else:
        return hamming_less_length_str(str2, str1)

In [26]:
def hamming_list_str_dist(input1, input2):
    '''
    input:
        input1: string or list of strings
        input2: string or list of strings
    output:
        the hamming distance
    '''
    if ( str(type(input1)) == "<class 'str'>"):
        return hamming_str(input1, input2)
    else:
        dist_sum = 0
        if (len(input1) == len(input2)):
            for i in range(0, len(input1)):
                dist_sum += hamming_str(input1[i], input2[i])
            return dist_sum
        else:
            assert False, "Lists with different size, Kamaaal h bai!"

In [27]:
def LCS_str(str1, str2):
    n = len(str1)
    m = len(str2)
    
    LCS = np.zeros((n+1, m+1, 1)).astype('uint8')
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            if(str1[i-1] == str2[j-1]):
                LCS[i][j] = LCS[i-1][j-1]+1
            else:
                LCS[i][j] = max(LCS[i-1][j], LCS[i][j-1])
    
    a = n
    b = m
    sequence = ""
    if  (str(type(str1)) == "<class 'str'>"):
        sequence = ""
    else:
        sequence = []
    
    while(LCS[a][b][0] != 0):
        if(str1[a-1] == str2[b-1]):
            if  (str(type(str1)) == "<class 'str'>"):
                sequence += str1[a-1]
            else:
                sequence.append(str1[a-1])
            a -= 1
            b -= 1
        else:
            if(LCS[a-1][b][0] == LCS[a][b][0]):
                a -= 1
            else:
                b -= 1
    out  = {}
    out["size"] = LCS[n][m][0]
    out["sequence"] = sequence[::-1]
    return out

In [28]:
def LCS (input1, input2, base = 'char'):
    '''
    input:
        input1: string or list of strings
        input2: string or list of strings
    output:
        out: a dict object containing,
        'size' - the LCS size
        'sequence' - the LCS
    '''
    if(base == 'char'):
        if (str(type(input1)) == "<class 'str'>"):
            return LCS_str(input1, input2)
        else:
            size = 0
            sequence = []
            if(len(input1) < len(input2)):
                for i in range(0, len(input1)):
                    temp = LCS_str(input1[i], input2[i])
                    size += temp["size"]
                    sequence.append(temp["sequence"])
            else:
                for i in range(0, len(input2)):
                    temp = LCS_str(input1[i], input2[i])
                    size += temp["size"]
                    sequence.append(temp["sequence"])
            out = {}
            out["size"] = size
            out["sequence"] = sequence
            return out
    else:
        return LCS_str(input1, input2)